In [1]:
# Libraries
import numpy as np   
from sympy.utilities.lambdify import lambdify
from sympy.abc import x
from sympy import sin, cos, integrate

# Mavillan's function: creates ill-conditioned matrix and a random vector b
def mavillan_f(n):
    A = np.random.rand(n,n)
    b = np.random.rand(n)
    return (A,b)

# Definitions
A,b = mavillan_f(2)
alpha = 0.01
M = A + alpha*np.identity(b.shape[0])
it = 100

In [2]:
# GMRes without optimization
def gmres(A,b,it=100, tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
                
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [3]:
# GMres with preconditioner proposed by the paper. Bad implementation
# TODO: BORRAR EL M^-1, EL PROFE NOS MATARA!
def paper_bad_prec_gmres(A,b,M,it=100, tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    Z = np.zeros((b.shape[0], it+1))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    M_1 = np.linalg.inv(M)
    i = 0
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        Z[:,i] = np.dot(M_1,Q[:,i])
        w = np.dot(A,Z[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        error_i = np.linalg.norm(b - np.dot(A, np.dot(Z[:,:i+1], y)))
        
        if H[i+1,i] == 0 or error_i < tol:
            break
    return np.dot(Z[:,:i+1], y),i

In [4]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    # Left preconditioner -> A and b changes
    b = gmres(M,b,it=it,tol=tol)[0]
    for i in range(b.shape[0]):
        A[:,i] = gmres(M, A[:,i],it=it,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [44]:
# A better implementation of preconditioned GMRes without Cauchy integral (right)
def right_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        x = gmres(M, Q[:,i],it=it,tol=tol)[0]
        w = np.dot(A, x)
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    return gmres(M, x_tild,it=it,tol=tol)[0], i

In [45]:
testcases = ['normal','cancer','left','right','']
test = testcases[4]
error = 1e-10

if test == 'normal':
    print gmres(A,b,tol=error)
elif test == 'cancer':
    print paper_bad_prec_gmres(A,b,M)
elif test == 'right':
    print right_prec_gmres(A,b,M,tol=error)
elif test == 'left':
    print left_prec_gmres(A,b,M,tol=error)
else:
    print gmres(A,b,tol=error, it=it)
    print left_prec_gmres(A,b,M,tol=error, it=it)
    print right_prec_gmres(A,b,M,tol=error, it=it)
    print np.linalg.solve(A,b)

(array([-0.30143305,  2.21914349]), 19)
(array([-0.30142777,  2.21914512]), 7)
(array([-0.3014598 ,  2.21918023]), 5)
[-0.3014372   2.21914253]


In [9]:
def trapezoid(myfun, N, a, b):
    f = np.vectorize(myfun) # So we can apply it to arrays without trouble
    x = np.linspace(a, b, N+1) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    print f(x[0]).shape, f(xmiddle).shape, f(x[1]).shape
    int_val = 0.5*h*sum(f(x[0])+2*f(xmiddle)+f(x[-1]))

    return int_val

def contour_integral(A, v, f):
    lamb, vect = np.linalg.eig(A)
    M = np.amax(np.real(lamb))
    m = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    center = (M+m)/2
    a = center - m
    ide = np.identity(A.shape[0])
    
    reg = lambda x: a*sin(x) + 1j*d*cos(x)
    
    
    arg = lambda x: (reg(x) - center)*f(reg(x))*gmres(x*ide-A,v)[0]
    
    print "aqui"
    inte = trapezoid(arg, 10, 0, 2*np.pi)
    print "llegue"
    #inte =  integrate(arg(x),(x,0,2*np.pi))
    
    

f = 1/x
f = lambdify(x,f)
#contour_integral(A,b,f)
    

In [43]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(a, b, N+1) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += myfun(i)
    int_val = myfun(0) + 2*int_val + myfun(x[-1])
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,M,v):
    fz = 1./f(z(t, (l+L)/2., (L-l)/2.))
    dzz = 1./dz(t, (L-l)/2.)
    p = fz*dzz
    gmr = gmres(p*z(t, (l+L)/2., (L-l)/2.)*np.identity(v.shape[0]) - p*M, v, tol=1e-10)[0]
    return gmr

def cauchy_integral(l, L, M, v):
    N = 50
    f = lambda x: (1. - (alpha/x)**(N+1))/(x - alpha)
    g1 = lambda t: g(t,l,L,M,v)
    return trapezoid2(g1, 50, 0, 2*np.pi) / (2.*np.pi)

# GMRes using contour integral to compute the preconditioner
def cauchy_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    lamb, vect = np.linalg.eig(A)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        x = cauchy_integral(l, L, M, Q[:,i])
        w = np.dot(A, x)
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    x = cauchy_integral(l, L, M, x_tild)
    return x,i

print cauchy_prec_gmres(A, b, M, it=it, tol=error)


(array([-0.30412015,  2.22571431]), 22)


/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
/Library/Python/2.7/site-packages/ipykernel/__main__.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
/Library/Python/2.7/site-packages/ipykernel/__main__.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
